In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import sys
import pathlib
import os
from skmap.catalog import DataCatalog
# from skmap.loader import TiledDataLoader
from skmap.overlay import SpaceOverlay, SpaceTimeOverlay
from skmap.misc import find_files, GoogleSheet, ttprint
from osgeo.gdal import BuildVRT, SetConfigOption
import random
import pandas as pd
import time
import skmap_bindings as sb
import numpy as np
from shapely.geometry import Point
from geopandas import gpd 

version = 'v20250521'
ttprint('read')
ipnt = gpd.read_file(f'pnt_agg_all.{version}.gpkg')
ttprint('finish')
print(ipnt.shape)


[08:09:50] read
[08:11:22] finish
(744291, 9)


In [2]:
ml = []
years = [2009,2015,2018]
for tt in years:
    tpnt = ipnt.copy()
    tpnt['time'] = tt
    ml.append(tpnt)
    
pnt = pd.concat(ml)


In [3]:
base_path = [f'http://192.168.49.{gaia_id}:8333' for gaia_id in range(30,47)]
GDAL_OPTS = {'GDAL_HTTP_VERSION': '1.0', 'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': '.tif'}
max_ram_mb = 750000
n_threads = 96

# read in gsheet
gsheet_key = '/mnt/apollo/stac/gaia-319808-913d36b5fca4.json'
gsheet_url = 'https://docs.google.com/spreadsheets/d/1lNTpzdHBG5dirYj46iBDRJMk_YAV0Um2ovBc8v3dR9w/edit?gid=78425683#gid=78425683'
gsheet = GoogleSheet(gsheet_key, gsheet_url, verbose=False)

# create catalog
catalog = DataCatalog.create_catalog(catalog_def=gsheet.snr_selected, years=years, base_path=base_path) #, replace_group_feat_name=True
catalog.save_json(f'overlay_agg.srs_{version}.json')

In [4]:
years

[2009, 2015, 2018]

In [5]:
start = time.time()
space_overlay = SpaceTimeOverlay(
        col_date='time',
        points=pnt, 
        catalog=catalog,
        raster_tiles='./ard2_final_status.gpkg',
        verbose=True,
        n_threads=n_threads,
        tile_id_col='TILE')

print(f"Extraction of overlay meta-data: {(time.time() - start):.2f} s")


[08:11:47] Overlay 744291 points from 2009 in 73 raster layers
[08:11:47] Reading ./ard2_final_status.gpkg
[08:11:50] 0 out of 73 URLs returning 404
[08:11:54] Scanning blocks of 73 layers
[08:11:54] Finding query pixels for 0eedd36ec93fd9c99387cc8f5c320801 (1 layers)
[08:11:58] Finding query pixels for 1111cac5fd6669660413a5daaf395e08 (3 layers)
[08:12:01] Finding query pixels for 2fbfe950e4c22a02ba7e3b52884be34f (1 layers)
[08:12:05] Finding query pixels for 4febf6577f1f162c52ad482e075d0c76 (2 layers)
[08:12:14] Finding query pixels for 56140668da1ff98e935fb06418a942f2 (1 layers)
[08:12:19] Finding query pixels for 660960b8a39c6afeae9171be064ae057 (18 layers)
[08:12:28] Finding query pixels for 67d1c8ae1f1ff92ae4147038afb08e2b (3 layers)
[08:12:31] Finding query pixels for 6ed680f0705ae492bb83f45e130f7980 (1 layers)
[08:12:34] Finding query pixels for 725e180fc0b4da112983d7131869020a (3 layers)
[08:12:37] Finding query pixels for 741e70ee94219b346a230b59d312a20d (4 layers)
[08:12:50]

In [6]:
start = time.time()
ovelayed_data = space_overlay.run(gdal_opts=GDAL_OPTS, max_ram_mb=max_ram_mb, out_file_name=f"agg.pnt.tmp_overlaid_{version}.pq")
print(f"Reading overlayed layers: {(time.time() - start):.2f} s")
print(f'data size: ', ovelayed_data.shape)


[08:17:50] Running the overlay for 2009
[08:17:51] Loading and sampling 1 raster layers for group 0eedd36ec93fd9c99387cc8f5c320801


OMP: Warning #96: Cannot form a team with 73 threads, using 64 instead.
OMP: Hint Consider unsetting KMP_DEVICE_THREAD_LIMIT (KMP_ALL_THREADS), KMP_TEAMS_THREAD_LIMIT, and OMP_THREAD_LIMIT (if any are set).


[08:17:51] Loading and sampling 3 raster layers for group 1111cac5fd6669660413a5daaf395e08
[08:17:52] Loading and sampling 1 raster layers for group 2fbfe950e4c22a02ba7e3b52884be34f
[08:17:55] Loading and sampling 2 raster layers for group 4febf6577f1f162c52ad482e075d0c76
[08:17:58] Loading and sampling 1 raster layers for group 56140668da1ff98e935fb06418a942f2
[08:18:05] Loading and sampling 18 raster layers for group 660960b8a39c6afeae9171be064ae057
[08:18:27] Loading and sampling 3 raster layers for group 67d1c8ae1f1ff92ae4147038afb08e2b
[08:18:28] Loading and sampling 1 raster layers for group 6ed680f0705ae492bb83f45e130f7980
[08:18:28] Loading and sampling 3 raster layers for group 725e180fc0b4da112983d7131869020a
[08:18:29] Loading and sampling 4 raster layers for group 741e70ee94219b346a230b59d312a20d
[08:18:31] Loading and sampling 8 raster layers for group 82b14127aedf4840b02b3a63f38f9d3a
[08:18:31] Loading and sampling 1 raster layers for group 88cae960bc1da66c8a769730a4a4004

In [7]:
ovelayed_data

id nuts0  50km  100km  200km  id_50km  id_100km  id_200km  time  \
0           322    DE     1      0      0    280.0      93.0      32.0  2009   
1           323    DE     1      0      0    280.0      93.0      32.0  2009   
2           324    DE     1      0      0    280.0      93.0      32.0  2009   
3           325    DE     1      0      0    280.0      93.0      32.0  2009   
4           326    DE     1      0      0    280.0      93.0      32.0  2009   
...         ...   ...   ...    ...    ...      ...       ...       ...   ...   
2232868  861776    ES     1      0      0    264.0      83.0      27.0  2018   
2232869  861777    ES     1      0      0    264.0      83.0      27.0  2018   
2232870  861778    ES     1      0      0    264.0      83.0      27.0  2018   
2232871  861779    ES     1      0      0    264.0      83.0      27.0  2018   
2232872  861780    ES     1      0      0    264.0      83.0      27.0  2018   

         wv_mcd19a2v061.seasconv.m.yearly_p50_1km_s_YYYY0101_YYYY1231_go_epsg.4326_v20230619  \
0                                                   1268.0                                     
1                                                   1254.0                                     
2                                                   1219.0                                     
3                                                   1245.0                                     
4                                                   1262.0                                     
...                                                    ...                                     
2232868                                             1758.0                                     
2232869                                             1753.0                                     
2232870                                             1713.0                                     
2232871                                             1593.0                                     
2232872                                             1390.0                                     

         ...  CHELSA_pr_10_1981-2010_V.2.1  CHELSA_rsds_1981-2010_max_V.2.1  \
0        ...                         783.0                          18215.0   
1        ...                         791.0                          18219.0   
2        ...                         794.0                          18261.0   
3        ...                         792.0                          18324.0   
4        ...                         786.0                          18356.0   
...      ...                           ...                              ...   
2232868  ...                         775.0                          26122.0   
2232869  ...                         768.0                          26123.0   
2232870  ...                         761.0                          26127.0   
2232871  ...                         755.0                          26136.0   
2232872  ...                         749.0                          26139.0   

         max.curv.bareearth_ensemble_m_120m_s_20000101_20221231_eu_epsg.3035_v20240501  \
0                                                    -11.0                               
1                                                     82.0                               
2                                                     49.0                               
3                                                     69.0                               
4                                                     19.0                               
...                                                    ...                               
2232868                                               22.0                               
2232869                                                2.0                               
2232870                                               39.0                               
2232871                                           